## DNA Search 
Genes are commonly represented in computer software as a sequence of the characters
A, C, G, and T. Each letter represents a nucleotide, and the combination of
three nucleotides is called a codon. A codon codes
for a specific amino acid that together with other amino acids can form a protein.
A classic task in bioinformatics software is to find a particular codon within a gene.

In [1]:
from enum import IntEnum

Nucleotide = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
print(Nucleotide)

<enum 'Nucleotide'>


In [2]:
#Codon = (Nucleotide, Nucleotide, Nucleotide) # type alias for codons
#Gene = [Codon] # type alias for genes

In [3]:
gene_str = "ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT"

def string_to_gene(s):
    gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s): # don't run off end!
            return gene
        
        # initialize codon out of three nucleotides
        codon = (Nucleotide[s[i]], Nucleotide[s[i + 1]], Nucleotide[s[i + 2]])
        gene.append(codon) # add codon to gene
    return gene

In [4]:
my_gene = string_to_gene(gene_str)
print(my_gene)

[(<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>), (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.T: 4>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>), (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.C: 2>), (<Nucleotide.C: 2>, <Nucleotide.C: 2>, <Nucleotide.T: 4>), (<Nucleotide.A: 1>, <Nucleotide.G: 3>, <Nucleotide.G: 3>), (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.T: 4

### Linear Search

In [5]:
def linear_contains(gene, key_codon):
    for codon in gene:
        if codon == key_codon:
            return True
    return False

In [6]:
acg = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains(my_gene, acg)) # True
print(linear_contains(my_gene, gat)) # False

True
False


### Binary Search

In [7]:
def binary_contains(gene, key_codon):
    low = 0
    high = len(gene) - 1
    while low <= high: # while there is still a search space
        mid = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False

In [8]:
my_sorted_gene = sorted(my_gene)
print(binary_contains(my_sorted_gene, acg)) # True
print(binary_contains(my_sorted_gene, gat)) # False

True
False


In [9]:
print(binary_contains(["a", "d", "e", "f", "z"], "f")) # True
print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila")) # False

True
False


## Maze solving
Our maze will be a two-dimensional grid of Cells. A Cell is an enum with str values
where " " will represent an empty space and "X" will represent a blocked space.
There are also other cases for illustrative purposes when printing a maze.

In [12]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random

class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"

In [21]:
class MazeLocation(NamedTuple):
    row: int
    column: int

In [288]:
class Maze:
    def __init__(self, rows = 10, columns = 10, sparseness = 0.2, start = MazeLocation(0, 0), goal = MazeLocation(9, 9)):
        
        # initialize basic instance variables
        self._rows = rows
        self._columns = columns
        self.start = start
        self.goal = goal
        
        # fill the grid with empty cells
        self._grid = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        
        # populate the grid with blocked cells
        self._randomly_fill(rows, columns, sparseness)
        
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL
    
    def _randomly_fill(self, rows, columns, sparseness):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED

    # return a nicely formatted version of the maze for printing
    def __str__(self):
        output = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output

    def goal_test(self, ml):
        return ml == self.goal

    def successors(self, ml):
        locations = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != \
            Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != \
            Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != \
            Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    def mark(self, path):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

    def clear(self, path):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

In [ ]:
class Node:
    def __init__(self, state, parent, cost = 0.0, heuristic = 0.0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic
        
    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

In [ ]:
def node_to_path(node):
    path = [node.state]
    
    # work backwards from end to front
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    path.reverse()
    return path

### DFS

In [289]:
class Stack:
    def __init__(self):
        self._container = []
    
    @property
    def empty(self):
        return not self._container
    
    def push(self, item):
        self._container.append(item) # O(1)
    
    def pop(self):
        return self._container.pop() # O(1)
    
    def __repr__(self):
        return repr(self._container)

In [291]:
def dfs(initial, goal_test, successors):
    
    # frontier is where we've yet to go
    frontier = Stack()
    frontier.push(Node(initial, None))
    
    # explored is where we've been
    explored = {initial}
    
    # keep going while there is more to explore
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
    
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
    
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored: # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None # went through everything and never found goal

### BFS

In [293]:
from collections import deque

class Queue:
    def __init__(self):
        self._container = deque()
    
    @property
    def empty(self):
        return not self._container
    
    def push(self, item):
        self._container.append(item) # O(1)
        
    def pop(self):
        return self._container.popleft() # O(1)
    
    def __repr__(self):
        return repr(self._container)

In [294]:
def bfs(initial, goal_test, successors):
    
    # frontier is where we've yet to go
    frontier = Queue()
    frontier.push(Node(initial, None))
    
    # explored is where we've been
    explored = {initial}
    
    # keep going while there is more to explore
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state

        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node

        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored: # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    
    return None # went through everything and never found goal

In [302]:
m = Maze()
print(m)

SX        
         X
          
          
X     X   
X X  X  X 
    X  X  
  X    X  
      X   
         G



In [303]:
# Test DFS
solution1 = dfs(m.start, m.goal_test, m.successors)
if solution1 is None:
    print("No solution found using depth-first search!")
else:
    path1 = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

SX        
*********X
        **
 ******* *
X*    X***
X*X  X  X 
**  X  X  
* X    X  
******X   
     ****G



In [304]:
# Test BFS
solution2 = bfs(m.start, m.goal_test, m.successors)
if solution2 is None:
    print("No solution found using breadth-first search!")
else:
    path2 = node_to_path(solution2)
    m.mark(path2)
    print(m)
    m.clear(path2)

SX        
*        X
*         
**        
X*    X   
X*X  X  X 
 *  X  X  
 *X    X  
 *    X   
 ********G



### A* Algorithm (cost function + heuristic function [f(n)= g(n) + h(n)])

In [305]:
from heapq import heappush, heappop

class PriorityQueue():
    def __init__(self):
        self._container = []
    
    @property
    def empty(self):
        return not self._container # not is true for empty container
    
    def push(self, item):
        heappush(self._container, item) # in by priority O(log n)
    
    def pop(self):
        return heappop(self._container) # out by priority O(log n)
    
    def __repr__(self):
        return repr(self._container)

In [306]:
def manhattan_distance(goal):
    def distance(ml):
        xdist = abs(ml.column - goal.column)
        ydist = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

In [307]:
def astar(initial, goal_test, successors, heuristic):
    
    # frontier is where we've yet to go
    frontier = PriorityQueue()
    
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    
    # explored is where we've been
    explored = {initial: 0.0}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node = frontier.pop()
        current_state = current_node.state
        
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
        
        # check where we can go next and haven't explored
        for child in successors(current_state):
            new_cost = current_node.cost + 1 # 1 assumes a grid, need 
                            # a cost function for more sophisticated apps
            
            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost,
                heuristic(child)))
    
    return None # went through everything and never found goal

In [309]:
# Test A* algorithm
distance = manhattan_distance(m.goal)
solution3 = astar(m.start, m.goal_test, m.successors, distance)
if solution3 is None:
    print("No solution found using A*!")
else:
    path3 = node_to_path(solution3)
    m.mark(path3)
    print(m)

SX        
*        X
*         
**        
X*    X   
X*X  X  X 
 ***X  X  
  X*** X  
     *X   
     ****G

